## Read namespaces

In [1]:
# Read namespaces
import os
import kubernetes
import ipywidgets as widgets
from IPython.display import display, JSON

ns_list = []
kubernetes.config.load_kube_config()
k8s_client = kubernetes.client.CoreV1Api()
namespaces = k8s_client.list_namespace(_request_timeout=(1, 5))
for namespace in namespaces.items:
    ns_name = namespace.metadata.name
    ns_list.append(ns_name)
w = widgets.Dropdown(
    options=ns_list,
    description='SUT ns',
    disabled=False,
    layout=widgets.Layout(width='50%')
)
display(w)

Dropdown(description='SUT ns', layout=Layout(width='50%'), options=('binderhub', 'calico-apiserver', 'calico-o…

## Set namespaces

In [2]:
# Set namespaces
SUT_NAMESPACE = w.value
# print(f"Using SUT namespace {SUT_NAMESPACE}")
BRANCH_NAME = SUT_NAMESPACE.replace("ci-ska-mid-itf-", "")
# print(f"Using SUT branch {BRANCH_NAME}")
SKA001_NAMESPACE = f"ci-dish-lmc-ska001-{BRANCH_NAME}"
# print(f"Using SKA 001 namespace {SKA001_NAMESPACE}")
ska001 = widgets.Text(
    value=SKA001_NAMESPACE,
    description='SKA001 ns',
    disabled=True,
    layout=widgets.Layout(width='60%')
)
display(ska001)
SKA036_NAMESPACE = f"ci-dish-lmc-ska036-{BRANCH_NAME}"
ska036 = widgets.Text(
    value=SKA036_NAMESPACE,
    description='SKA036 ns',
    disabled=True,
    layout=widgets.Layout(width='60%')
)
display(ska036)
# print(f"Using SKA 036 namespace {SKA036_NAMESPACE}")

# print(f"Setting up parameters for SUT namespace {SUT_NAMESPACE}\n")

TANGO_HOST = f"tango-databaseds.{SUT_NAMESPACE}.svc.miditf.internal.skao.int:10000"
os.environ["TANGO_HOST"] = TANGO_HOST

th = widgets.Text(
    value=os.getenv("TANGO_HOST"),
    description='Tango host',
    disabled=True,
    layout=widgets.Layout(width='80%')
)
display(th)

Text(value='ci-dish-lmc-ska001-at-2146-consolidate-jupyter-notebooks', description='SKA001 ns', disabled=True,…

Text(value='ci-dish-lmc-ska036-at-2146-consolidate-jupyter-notebooks', description='SKA036 ns', disabled=True,…

Text(value='tango-databaseds.ci-ska-mid-itf-at-2146-consolidate-jupyter-notebooks.svc.miditf.internal.skao.int…

## Configure Device Proxies

In [3]:
# Set Up and Configure Device Proxies

RECEPTORS = ["SKA001", "SKA036"]

# Config files set up
DATA_DIR = "../../data"
TMC_CONFIGS = f"{DATA_DIR}/mid_telescope/tmc"
SCAN_FILE = f"{TMC_CONFIGS}/scan.json"
RELEASE_RESOURCES_FILE = f"{DATA_DIR}/release_resources.json"

ASSIGN_RESOURCES_FILE = f"{TMC_CONFIGS}/assign_resources.json"
CONFIGURE_SCAN_FILE = f"{TMC_CONFIGS}/configure_scan.json"

CBF_CONFIGS = f"{DATA_DIR}/mid_telescope/cbf"
DISH_CONFIG_FILE = f"{CBF_CONFIGS}/sys_params/load_dish_config.json"

# print("Links for Taranta and QA Display\n")
print("SUT Links")
print(f"https://k8s.miditf.internal.skao.int/{SUT_NAMESPACE}/signal/display/")
print(f"https://k8s.miditf.internal.skao.int/{SUT_NAMESPACE}/taranta/devices")
print("\n")
print("Dish LMC links")
print(f"https://k8s.miditf.internal.skao.int/{SKA001_NAMESPACE}/taranta/devices")
print(f"https://k8s.miditf.internal.skao.int/{SKA036_NAMESPACE}/taranta/devices")

SUT Links
https://k8s.miditf.internal.skao.int/ci-ska-mid-itf-at-2146-consolidate-jupyter-notebooks/signal/display/
https://k8s.miditf.internal.skao.int/ci-ska-mid-itf-at-2146-consolidate-jupyter-notebooks/taranta/devices


Dish LMC links
https://k8s.miditf.internal.skao.int/ci-dish-lmc-ska001-at-2146-consolidate-jupyter-notebooks/taranta/devices
https://k8s.miditf.internal.skao.int/ci-dish-lmc-ska036-at-2146-consolidate-jupyter-notebooks/taranta/devices


## Set Up Device Proxies

In [5]:
import json
import time
import tango

from tango import DeviceProxy

proxy_w = widgets.IntProgress(
    value=0,
    min=0,
    max=10,
    description='Proxies',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'maroon'},
    orientation='horizontal'
)
display(proxy_w)
proxy_ok = True
try:
    # TMC proxies
    proxy_w.value = 1
    tmc_central_node = DeviceProxy("ska_mid/tm_central/central_node")
    proxy_w.value = 2
    tmc_csp_master = DeviceProxy("ska_mid/tm_leaf_node/csp_master")
    proxy_w.value = 3
    tmc_csp_subarray = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray01")
    proxy_w.value = 4
    tmc_subarray = DeviceProxy("ska_mid/tm_subarray_node/1")
    # CSP.LMC proxies
    proxy_w.value = 5
    csp_control = DeviceProxy("mid-csp/control/0")
    proxy_w.value = 6
    csp_subarray = DeviceProxy("mid-csp/subarray/01")
    # CBF proxies
    proxy_w.value = 7
    cbf_controller = DeviceProxy("mid_csp_cbf/sub_elt/controller")
    proxy_w.value = 8
    cbf_subarray = DeviceProxy("mid_csp_cbf/sub_elt/subarray_01")
    # Dish Leaf Proxies
    proxy_w.value = 9
    dish_leaf_node_ska001 = DeviceProxy("ska_mid/tm_leaf_node/d0001")
    proxy_w.value = 10
    dish_leaf_node_ska036 = DeviceProxy("ska_mid/tm_leaf_node/d0036")
    # SDP Proxies
    proxy_w.value = 11
    sdp_subarray = DeviceProxy("mid-sdp/subarray/01")
except tango.DevFailed as terr:
    print(f"ERROR: {terr.args[0].desc.strip()}")
    tmc_central_node = None
    tmc_csp_master = None
    proxy_ok = False
        
if proxy_ok:
    print(f'Set up {proxy_w.value} device proxies')




IntProgress(value=0, description='Proxies', max=10, style=ProgressStyle(bar_color='maroon'))

Set up 10 device proxies


## Diagnostics

### Smoke test

In [6]:
# Smoke test
if tmc_central_node is None:
    print(f"ERROR: No TMC central none")
else:
    try:
        tping = tmc_central_node.ping()
        print(f"Ping TMC central node: {tping}")
    except tango.ConnectionFailed as terr:
        err_msg = terr.args[0].desc.strip()
        print(f"Tango connection failed: {err_msg}")
        tping = 0
if tping:
    tmc_admin = tmc_central_node.adminMode
else:
    tmc_admin = -1
    
tmc_cent_w = widgets.Valid(
    value=not tmc_admin,
    description="TMC Central Node online",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(tmc_cent_w)

Ping TMC central node: 21900


Valid(value=False, description='TMC Central Node online', layout=Layout(width='60%'), style=DescriptionStyle(d…

### Print TMC diagnostics

In [7]:
# Print TMC diagnostics
try:
    print(f"| TMC Central Node           | {tmc_central_node.name():40} |")
    print(f"| TMC Central Node state     | {str(tmc_central_node.State()):40} |")
    print(f"| TMC Central Node adminMode | {str(tmc_central_node.adminmode):40} |")
    # print(f"TMC Central Node healthState: {str(tmc_central_node.health_state)}")
    # print(f"TMC Central Node telescopeHealthState: {str(tmc_central_node.telescope_health_state)}")
    # print(f"TMC Central Node isDishVccConfig: {str(tmc_central_node.isDishVccConfigSet)}")
    # print(f"TMC Central Node dishvccvalidationstatus: {str(tmc_central_node.dishvccvalidationstatus)}")
    # print(f"TMC Subarray Node state: {tmc_csp_subarray.State()}")
    # print(f"TMC Subarray adminMode: {str(tmc_csp_subarray.admin_mode)}")
    # print(f"TMC Subarray Node obsState: {str(tmc_csp_subarray.obs_state)}")
except tango.ConnectionFailed as terr:
    print(f"Tango connection failed: {terr.args[0].desc.strip()}")
# table(tmc_rows, frame=True)

| TMC Central Node           | ska_mid/tm_central/central_node          |
| TMC Central Node state     | ON                                       |
| TMC Central Node adminMode | adminMode.OFFLINE                        |


### Print CSP-LMC Diagnostics

In [8]:
# Print CSP-LMC Diagnostics
try:
    # Controller
    print(f"| CSP-LMC Controller                     | {csp_control.name():40} |")
    print(f"| CSP-LMC Controller admin mode          | {str(csp_control.adminMode):40} |")
    print(f"| CSP-LMC Controller State               | {csp_control.State():40} |")
    print(f"| CSP-LMC Controller dish Vcc Config     | {csp_control.dishVccConfig:40} |")
    print(f"| CSP-LMC Controller CBF Simulation Mode | {csp_control.cbfSimulationMode:40} |")
    # Subarray
    print(f"| CSP-LMC Subarray                       | {csp_subarray.name():40} |")
    print(f"| CSP-LMC Subarray admin mode            | {str(csp_subarray.adminMode):40} |")
    print(f"| CSP-LMC Subarray State                 | {csp_subarray.State():40} |")
    print(f"| CSP-LMC Subarray observation state     | {str(csp_subarray.obsState):40} |")
    print(f"| CSP-LMC Subarray dish Vcc Config       | {csp_subarray.dishVccConfig:40} |")
except tango.ConnectionFailed as terr:
    print(f"Tango connection failed: {terr.args[0].desc.strip()}")


| CSP-LMC Controller                     | mid-csp/control/0                        |
| CSP-LMC Controller admin mode          | adminMode.OFFLINE                        |
| CSP-LMC Controller State               |                                       12 |
| CSP-LMC Controller dish Vcc Config     |                                          |
| CSP-LMC Controller CBF Simulation Mode |                                        1 |
| CSP-LMC Subarray                       | mid-csp/subarray/01                      |
| CSP-LMC Subarray admin mode            | adminMode.OFFLINE                        |
| CSP-LMC Subarray State                 |                                       12 |
| CSP-LMC Subarray observation state     | obsState.EMPTY                           |
| CSP-LMC Subarray dish Vcc Config       |                                          |


### Print CBF Diagnostics

In [9]:
# Print CBF Diagnostics
try:
    # CBF Controller
    print(f"| CBF Controller admin mode      | {str(cbf_controller.adminMode):40} |")
    print(f"| CBF Controller State           | {str(cbf_controller.State()):40} |")
    # CBF Subarray
    print(f"| CBF Subarray admin mode        | {str(cbf_subarray.adminMode):40} |")
    print(f"| CBF Subarray State             | {str(cbf_subarray.State()):40} |")
    print(f"| CBF Subarray observation state | {str(cbf_subarray.obsState):40} |")
except tango.ConnectionFailed as terr:
    err_msg = terr.args[0].desc.strip()
    print("Tango connection failed: %s", err_msg)

| CBF Controller admin mode      | adminMode.OFFLINE                        |
| CBF Controller State           | DISABLE                                  |
| CBF Subarray admin mode        | adminMode.OFFLINE                        |
| CBF Subarray State             | DISABLE                                  |
| CBF Subarray observation state | obsState.EMPTY                           |


### Print SDP Diagnostics

In [10]:
# Print SDP Diagnostics
try:
    # print(f"SDP Controller state: {sdp_controller.State()}")
    # print(f"SDP Controller admin mode: {str(sdp_controller.admin_mode)}")
    # SDP Subarray
    print(f"SDP Subarray state             | {str(sdp_subarray.State()):40} |")
    print(f"SDP Subarray admin mode        | {str(sdp_subarray.adminMode):40} |")
    print(f"SDP Subarray observation state | {str(sdp_subarray.obsState):40} |")
except tango.ConnectionFailed as terr:
    err_msg = terr.args[0].desc.strip()
    print("Tango connection failed: %s", err_msg)

SDP Subarray state             | OFF                                      |
SDP Subarray admin mode        | adminMode.ONLINE                         |
SDP Subarray observation state | obsState.EMPTY                           |


## Set devices to ONLINE

In [11]:
# Set devices to adminMode = ONLINE
csp_control.adminMode = 0
csp_subarray.adminMode = 0

# Leaf Nodes
csp_subarray_leaf_node = "ska_mid/tm_leaf_node/csp_subarray01"
sdp_subarray_leaf_node = "ska_mid/tm_leaf_node/sdp_subarray01"
csp_master_leaf_node = "ska_mid/tm_leaf_node/csp_master"
csp_subarray_leaf_node_dp = DeviceProxy(csp_subarray_leaf_node)
sdp_subarray_leaf_node_dp = DeviceProxy(sdp_subarray_leaf_node)
csp_master_leaf_node_dp = DeviceProxy(csp_master_leaf_node)

time.sleep(2)
print("Admin mode after setting to ONLINE")

# CSP Control admin mode
#print(f"  CSP Control: {csp_control.adminMode}")
csp_ctl = widgets.Valid(
    value=not csp_control.adminMode,
    description="CSP Control online",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(csp_ctl)

# CSP Subarray admin mode
# print(f"  CSP Subarray: {csp_subarray.adminMode}")
csp_sub = widgets.Valid(
    value=not csp_subarray.adminMode,
    description="CSP Subarray online",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(csp_sub)

# CBF Controller admin mode
# print(f"  CBF Controller: {cbf_controller.adminMode}")
cbf_ctl = widgets.Valid(
    value=not cbf_controller.adminMode,
    description="CBF Controller online",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(cbf_ctl)

# CBF Subarray
# print(f"  CBF Subarray: {cbf_subarray.adminMode}")
cbf_sub = widgets.Valid(
    value=not cbf_subarray.adminMode,
    description="CBF Subarray online",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(cbf_sub)

# Set CBF Simulation mode to false and CBF timeout to 99s
csp_control.cbfSimulationMode = 0
csp_control.commandTimeout = 99

time.sleep(2)
# print("\nChecking CBF Simulation Mode and CBF Timeout:")

# CBF Simulation Mode
# print(f"  CBF Simulation Mode: {bool(csp_control.cbfSimulationMode)}")
cbf_sim = widgets.Valid(
    value=bool(csp_control.cbfSimulationMode),
    description="CBF Simulation Mode",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(cbf_sim)

# CBF Timeout
# print(f"  CBF Timeout: {csp_control.commandTimeout} sec")
cbf_tim = widgets.IntSlider(
    value=csp_control.commandTimeout,
    min=0,
    max=180,
    step=1,
    description='CBF Timeout',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(cbf_tim)

Admin mode after setting to ONLINE


Valid(value=True, description='CSP Control online', layout=Layout(width='60%'), style=DescriptionStyle(descrip…

Valid(value=True, description='CSP Subarray online', layout=Layout(width='60%'), style=DescriptionStyle(descri…

Valid(value=True, description='CBF Controller online', layout=Layout(width='60%'), style=DescriptionStyle(desc…

Valid(value=True, description='CBF Subarray online', layout=Layout(width='60%'), style=DescriptionStyle(descri…

Valid(value=False, description='CBF Simulation Mode', layout=Layout(width='60%'), style=DescriptionStyle(descr…

IntSlider(value=99, continuous_update=False, description='CBF Timeout', max=180)

## Load the Dish Vcc Config / Init Sys Params

In [13]:
# Load the Dish Vcc Config / Init Sys Params
from IPython.display import JSON
# csp_control.commandTimeout = cbf_tim.value

with open(DISH_CONFIG_FILE, encoding="utf-8") as f:
    dish_config_json = json.load(f)

dish_config_json["tm_data_sources"][0] = "car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata"
dish_config_json["tm_data_filepath"] = "instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json"
# print(f"Dish config json file contents: \n{dish_config_json}")

nmax = 2
cfgp_w = widgets.IntProgress(
    value=0,
    min=0,
    max=nmax,
    description='Loading:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'maroon'},
    orientation='horizontal'
)
display(cfgp_w)

# NOTE: RUN TWICE
for n in range(1, nmax+1):
    cfgp_w.value = n
    # print(f"\nLoad dish configuration {n}")
    try:
        tmc_central_node.LoadDishCfg(json.dumps(dish_config_json))
    except tango.ConnectionFailed as terr:
        err_msg = terr.args[0].desc.strip()
        print(f"Error: {err_msg}")
    time.sleep(2)

tmc_csp_dish_vcc = json.loads(tmc_csp_master.dishVccConfig)
# print(f"TMC CSP Master's Dish Vcc Config attribute value: \n"
# "{json.dumps(tmc_csp_master.dishVccConfig, indent=4)}")
print(f'TMC CSP Master Dish Vcc Config interface\n{tmc_csp_dish_vcc["interface"]}')
# json.dumps(tmc_csp_master.dishVccConfig, indent=4)
# cfgj = JSON(json.loads(tmc_csp_master.dishVccConfig), expanded=True)
# display(cfgj)

tmc_csp_src_vcc = json.loads(tmc_csp_master.sourceDishVccConfig)
# print(f"TMC CSP Master's Source Dish Vcc Config attribute value: \n"
# "{tmc_csp_master.sourceDishVccConfig}")
print(f'TMC CSP Master Source Dish Vcc Config interface\n{tmc_csp_src_vcc["interface"]}')


IntProgress(value=0, description='Loading:', max=2, style=ProgressStyle(bar_color='maroon'))

TMC CSP Master Dish Vcc Config interface
https://schema.skao.int/ska-mid-cbf-initsysparam/1.0
TMC CSP Master Source Dish Vcc Config interface
https://schema.skao.int/ska-mid-cbf-initsysparam/1.0


## Turn the Telescope On

### Run the Telescope On command

In [15]:
# Turn the Telescope On
import tango

print("Running the TelescopeOn command")
try:
    tmc_central_node.TelescopeOn()
except tango.DevFailed as t_err:
    err_msg: str = t_err.args[0].desc.strip()
    print(f"Could not run command: {err_msg}")
print("Ready")

Running the TelescopeOn command
Could not run command: ska_tmc_common.exceptions.CommandNotAllowed: Dish Vcc Config not Set. Please set using LoadDishCfg command. Current Telescope State is :ON
Ready


In [16]:
tel_on_retry = widgets.IntSlider(
    value=20,
    min=0,
    max=100,
    step=1,
    description='Retry',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(tel_on_retry)
tel_on_wait = widgets.IntSlider(
    value=5,
    min=0,
    max=30,
    step=1,
    description='Sleep',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(tel_on_wait)

IntSlider(value=20, continuous_update=False, description='Retry')

IntSlider(value=5, continuous_update=False, description='Sleep', max=30)

### Check if telescope is on

In [17]:
# Check if telescope is on
nmax = tel_on_retry.value
sleep_time = tel_on_wait.value
p = widgets.IntProgress(
    value=0,
    min=0,
    max=nmax,
    description='Loading:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'maroon'},
    orientation='horizontal'
)
display(p)
n = 0
while int(tmc_central_node.telescopeState) != 0:
    # print("Waiting for Telescope to come ON")
    n += 1
    p.value = n
    if n >= nmax:
        break
    try:
        time.sleep(sleep_time)
    except KeyboardInterrupt:
        print("Abort")
        break
    
tel_on = widgets.Valid(
    value=not tmc_central_node.telescopeState,
    description="Telescope ON",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(tel_on)
# if int(tmc_central_node.telescopeState) == 0:
#     print("Telescope is ON")
# else:
#    print("Telescope is not ON")

IntProgress(value=0, description='Loading:', max=10, style=ProgressStyle(bar_color='maroon'))

Valid(value=False, description='Telescope ON', layout=Layout(width='60%'), style=DescriptionStyle(description_…

# Assign Resources

In [18]:
# Assign Resources

print(f"SDP subarray is {sdp_subarray.state()}")
sdp_sub_w = widgets.Valid(
    value=sdp_subarray.state(),
    description="SDP subarray",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(sdp_sub_w)

time.sleep(3)

print(
    "Running the AssignResources command: subarray obsstate should go to Idle and receptor IDs should be assigned"
)

with open(ASSIGN_RESOURCES_FILE, encoding="utf-8") as f:
    assign_resources_json = json.load(f)
    assign_resources_json["dish"]["receptor_ids"] = RECEPTORS
    assign_resources_json["sdp"]["resources"]["receptors"] = RECEPTORS


# print(f"\nAssign_resources_json file contents: \n{assign_resources_json}")

try:
    tmc_subarray.AssignResources(json.dumps(assign_resources_json))
    res_ok = True
except tango.DevFailed as t_err:
    err_msg: str = t_err.args[0].desc.strip()
    print(f"Could not assign resources: {err_msg}")
    res_ok = False

time.sleep(2)
print(f"\nCBF Subarray Observation State: {tmc_subarray.obsState}")
print(f"CBF Subarray Receptors : {cbf_subarray.receptors}")

res_ok_w = widgets.Valid(
    value=res_ok,
    description="Resources",
    layout=widgets.Layout(width='60%'),
    style = {'description_width': 'initial'}
)
display(res_ok_w)

SDP subarray is ON


Valid(value=False, description='SDP subarray', layout=Layout(width='60%'), style=DescriptionStyle(description_…

Running the AssignResources command: subarray obsstate should go to Idle and receptor IDs should be assigned

CBF Subarray Observation State: 1
CBF Subarray Receptors : ()


Valid(value=True, description='Resources', layout=Layout(width='60%'), style=DescriptionStyle(description_widt…

## Deploy dishes

In [19]:
from astropy.time import Time
from tango import DeviceProxy

dish_deployments = [f"tango://tango-databaseds.{SKA001_NAMESPACE}.svc.miditf.internal.skao.int:10000/mid-dish/dish-manager/SKA001", 
                    f"tango://tango-databaseds.{SKA036_NAMESPACE}.svc.miditf.internal.skao.int:10000/mid-dish/dish-manager/SKA036"]


def get_tai_from_unix_s(unix_s: float) -> float:
    """
    Calculate atomic time in seconds from unix time in seconds.

    :param unix_s: Unix time in seconds

    :return: atomic time (tai) in seconds
    """
    astropy_time_utc = Time(unix_s, format="unix")
    return astropy_time_utc.unix_tai


for dish in dish_deployments:
    dish_manager_proxy = DeviceProxy(dish)
    current_pointing = dish_manager_proxy.achievedPointing
    current_az = current_pointing[1]
    current_el = current_pointing[2]

    current_time_tai_s = get_tai_from_unix_s(time.time() + 120)

    # Directions to move values
    az_dir = 1 if current_az < 350 else -1
    el_dir = 1 if current_el < 80 else -1

    track_table = [
        current_time_tai_s + 3,
        current_az + 1 * az_dir,
        current_el + 1 * el_dir,
        current_time_tai_s + 5,
        current_az + 2 * az_dir,
        current_el + 2 * el_dir,
        current_time_tai_s + 7,
        current_az + 3 * az_dir,
        current_el + 3 * el_dir,
        current_time_tai_s + 9,
        current_az + 4 * az_dir,
        current_el + 4 * el_dir,
        current_time_tai_s + 11,
        current_az + 5 * az_dir,
        current_el + 5 * el_dir,
    ]

    dish_manager_proxy.programTrackTable = track_table

ConnectionFailed: DevFailed[
DevError[
    desc = TRANSIENT CORBA system exception: TRANSIENT_NoUsableProfile
  origin = void Tango::Connection::connect(const string&) at (/src/cppTango/src/client/devapi_base.cpp:609)
  reason = API_CorbaException
severity = ERR]

DevError[
    desc = Failed to connect to database on host tango-databaseds.ci-dish-lmc-ska001-at-2146-consolidate-jupyter-notebooks.svc.miditf.internal.skao.int with port 10000
  origin = void Tango::Connection::connect(const string&) at (/src/cppTango/src/client/devapi_base.cpp:609)
  reason = API_CantConnectToDatabase
severity = ERR]
]

## Start Scan

### Configure Scan

In [ ]:
# Configure Scan
print("Running the Configure command: subarray obsstate should go to Ready")

with open(CONFIGURE_SCAN_FILE, encoding="utf-8") as f:
    configure_scan_json = json.load(f)

print(f"\nconfigure_scan_json file contents: \n{configure_scan_json}")

print(json.dumps(configure_scan_json))

tmc_subarray.Configure(json.dumps(configure_scan_json))
    
time.sleep(12)
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

### Start scan

In [ ]:
# TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING
with open("../../data/mid_telescope/tmc/scan.json", "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    sdp_scan = d["sdp"]
    print(sdp_scan)
    sdp_subarray_leaf_node_dp.scan(json.dumps(sdp_scan))

sleep(10)
print(sdp_subarray_leaf_node_dp.sdpSubarrayObsState)

with open("../../data/mid_telescope/tmc/scan.json", "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    csp_scan = d["csp"]
    print(csp_scan)
    csp_subarray_leaf_node_dp.scan(json.dumps(csp_scan))

sleep(2)
print(csp_subarray_leaf_node_dp.cspSubarrayObsState)

### End scan

In [ ]:
# TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING
print("Running the End Scan command: subarray obsstate should go to Ready state")

sdp_subarray_leaf_node_dp.EndScan()
sleep(2)
csp_subarray_leaf_node_dp.EndScan()
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

## End

### Run the End command

In [ ]:
# TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING
print("Running the End command: subarray obsstate should go to Idle state")

sdp_subarray_leaf_node_dp.End()
sleep(2)
csp_subarray_leaf_node_dp.End()

sleep(2)
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

### Release All Resources

In [ ]:
# TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING
print(
    "Running the Release All Resources command: subarray obsstate should go to Empty state and receptor IDs should be empty"
)

sdp_subarray_leaf_node_dp.ReleaseAllResources()
sleep(2)
csp_subarray_leaf_node_dp.ReleaseAllResources()

sleep(2)
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

### Run the Telescope Off command

In [ ]:
# Telescope Off
print("Running the TelescopeOff command")

tmc_central_node.TelescopeOff()

sleep(5)
print(f"TMC Central Node State: {tmc_central_node.State()}")
print(f"CSP Control State: {csp_control.State()}")
print(f"CBF Controller State: {cbf_controller.State()}")